In [7]:
# Shell Plus Model Imports
import os
import django
import datetime
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from book.models import Book, Metadata
from edu_book.models import EduBook, EduMetadata
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
from taggit.models import Tag, TaggedItem
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery

In [17]:
Metadata.objects.filter(market="yes24", crawl_date__year=2022, crawl_date__month=1, crawl_date__day=22).count()

0

In [22]:
EduBook.objects.all().delete()

(87559,
 {'edu_book.EduMetadata': 16069,
  'taggit.TaggedItem': 62817,
  'edu_book.EduBook': 8673})

In [18]:
Book.objects.filter(publisher="이지스퍼블리싱").count()

282

In [13]:
EduBook.objects.all().count()

3981

In [25]:
Book.objects.get(isbn="9791163030911")

DoesNotExist: Book matching query does not exist.

In [3]:
from collections import Counter
import re
from functools import reduce

In [3]:
tagsPart1 = Book.objects.all()[0].tags.names()
tagsPart2 = Book.objects.all()[1].tags.names()

print(tagsPart1, len(tagsPart1))
print(tagsPart2, len(tagsPart2))
print(tagsPart1.union(tagsPart2, all=True), len(tagsPart1.union(tagsPart2, all=True)))

<QuerySet ['#개발자', '#두잇', '#분철', '#새해결심', '#컴공', '#파이썬', '#프로그래밍', 'IT 모바일', '국내도서', '파이썬', '프로그래밍 언어']> 11
<QuerySet ['#다시보는2020', '2020년 올해의 책 후보도서', 'IT 모바일', 'YES24 올해의 책', '국내도서', '컴퓨터 공학', '컴퓨터 교육']> 7
<QuerySet ['#개발자', '#두잇', '#분철', '#새해결심', '#컴공', '#파이썬', '#프로그래밍', 'IT 모바일', '국내도서', '파이썬', '프로그래밍 언어', '#다시보는2020', '2020년 올해의 책 후보도서', 'IT 모바일', 'YES24 올해의 책', '국내도서', '컴퓨터 공학', '컴퓨터 교육']> 18


In [42]:
totalTags = {}
tidyTagsPart1 = list(set(" ".join(list(tagsPart1)).replace("#", "").split(" ")))
tidyTagsPart1.remove("모바일")
tidyTagsPart1.remove("IT")
tidyTagsPart1.remove("국내도서")
for item in tidyTagsPart1:
    try: 
        totalTags[item] = totalTags[item] + 1
    except:
        totalTags[item] = 1

totalTags

{'새해결심': 1, '프로그래밍': 1, '파이썬': 1, '언어': 1, '컴공': 1, '두잇': 1, '분철': 1, '개발자': 1}

In [51]:
totalTags = {}
for book in Book.objects.all():
    tidyTagsPart = list(set(" ".join(list(book.tags.names())).replace("#", "").split(" ")))
    
    try:
        tidyTagsPart.remove("모바일")
    except:
        pass
    
    try:
        tidyTagsPart.remove("IT")
    except:
        pass
    
    try:
        tidyTagsPart.remove("국내도서")
    except:
        pass
    
    try:
        tidyTagsPart.remove("컴퓨터")
    except:
        pass
    
    try:
        tidyTagsPart.remove("프로그래밍")
    except:
        pass
    
    try:
        tidyTagsPart.remove("컴공")
    except:
        pass
    
    try:
        tidyTagsPart.remove("공학")
    except:
        pass
    
    for item in tidyTagsPart:
        try: 
            totalTags[item] = totalTags[item] + 1
        except:
            totalTags[item] = 1

In [60]:
totalTags
dict(sorted(totalTags.items(), key=lambda x: x[1], reverse=True))

{'언어': 781,
 'OS/데이터베이스': 356,
 '파이썬': 298,
 '그래픽/디자인/멀티미디어': 297,
 '기타': 261,
 '클라우드/빅데이터': 250,
 '인공지능': 238,
 '일반': 234,
 '입문/활용': 228,
 '교육': 223,
 '분철': 212,
 '자료구조/알고리즘': 209,
 '활용': 185,
 '오피스': 185,
 '소프트웨어': 168,
 '웹사이트': 167,
 '어린이': 151,
 '네트워크/해킹/보안': 144,
 '대학교재': 142,
 'MS': 137,
 '인터넷': 133,
 '경영': 133,
 '경제': 133,
 '개발방법론': 118,
 '공학계열': 111,
 '비즈니스': 104,
 '컴퓨터공학': 102,
 '모바일/태블릿/SNS': 100,
 '게임': 100,
 '그래픽일반/자료집': 97,
 'C': 87,
 'Excel(엑셀)': 85,
 '보안/해킹': 84,
 '자바': 79,
 '데이터베이스': 79,
 '시스템관리/서버': 75,
 '마케팅': 70,
 'HTML/JavaScript/CSS/jQuery': 69,
 '네트워크': 64,
 '개발자': 61,
 '개발': 60,
 '마케팅/세일즈': 59,
 'UI/UX': 59,
 '예술': 59,
 'JavaScript/CGI': 58,
 '포토샵': 56,
 'PowerPoint(파워포인트)': 54,
 '인터넷비즈니스': 54,
 '프레젠테이션': 51,
 '안드로이드폰': 51,
 '미술': 50,
 '그래픽툴/저작툴': 49,
 '위한': 47,
 '음악/영상': 47,
 '멀티미디어/컴퓨터': 47,
 '자연과학': 46,
 '어른을': 44,
 'C++': 43,
 '쇼핑몰/창업': 40,
 '자기계발': 40,
 'AUTOCAD': 40,
 '': 39,
 'IT스테디셀러': 39,
 '입문서': 38,
 '통신': 37,
 '네트워크/데이터': 37,
 '웹디자인': 36,
 '리눅스': 36,
 

In [67]:
totalTagsList = []
for book in Book.objects.all():
    totalTagsList.append("|".join(list(book.tags.names())).replace("#", ""))
    
totalTagsList

['개발자|두잇|분철|새해결심|컴공|파이썬|프로그래밍|IT 모바일|국내도서|파이썬|프로그래밍 언어',
 '다시보는2020|2020년 올해의 책 후보도서|IT 모바일|YES24 올해의 책|국내도서|컴퓨터 공학|컴퓨터 교육',
 '2019IT연말결산선정도서|개발자|분철|새해결심|집콕공부|컴공|파이썬|프로그래머|혼공|2019년 올해의 책 후보도서|IT 모바일|YES24 올해의 책|국내도서|파이썬|프로그래밍 언어',
 '분철|IT 모바일|MS Excel(엑셀)|MS PowerPoint(파워포인트)|MS Word(워드)|국내도서|오피스 활용|한글',
 'IT 모바일|OS/데이터베이스|국내도서|인공지능|자료구조/알고리즘|컴퓨터 공학|클라우드/빅데이터|파이썬|프로그래밍 언어',
 'IT 모바일|국내도서|모바일/태블릿/SNS|쇼핑몰/창업|앱가이드|인터넷 마케팅|인터넷 비즈니스',
 '분철|IT 모바일|국내도서|그래픽/디자인/멀티미디어|일러스트레이터|포토샵',
 '1인창업|분철|온라인쇼핑몰창업|e-비즈니스|IT 모바일|경제 경영|국내도서|마케팅/세일즈|쇼핑몰/창업|인터넷 마케팅|인터넷 비즈니스|인터넷비즈니스',
 '2019IT연말결산선정도서|MS오피스|분철|사회초년생주목|새해결심|신입사원|일잘러업무스킬|집콕공부|칼퇴|IT 모바일|MS Excel(엑셀)|국내도서|오피스 활용',
 '분철|IT 모바일|각국 경제/경제사/전망|경제|경제 경영|경제전망|국내도서|국제경제/금융/화폐|네트워크/해킹/보안|마케팅/세일즈|보안/해킹|블록체인|컴퓨터 공학|트렌드/미래예측',
 '금융공학|데이터분석|퀀트|IT 모바일|OS/데이터베이스|국내도서|인공지능|자료구조/알고리즘|컴퓨터 공학|클라우드/빅데이터|파이썬|프로그래밍 언어',
 'C언어|개발자|분철|컴공|코딩|프로그래머|혼공|C|IT 모바일|국내도서|프로그래밍 언어',
 'SQL|국가공인|스펙|자격증|취준|IT 모바일|OS/데이터베이스|SQL Sever|국내도서|신규자격증/기타|컴퓨터 수험서',
 '분철|삼성코딩테스트준비|코딩테스트|IT 모바일|

In [68]:
totalTagsString = "|".join(totalTagsList)
totalTagsString = totalTagsString.replace(" ", "ssppaaccee")
totalTagsString = totalTagsString.replace("/", "and")
totalTagsString = totalTagsString.replace("-", "to")
totalTagsString

'개발자|두잇|분철|새해결심|컴공|파이썬|프로그래밍|ITssppaaccee모바일|국내도서|파이썬|프로그래밍ssppaaccee언어|다시보는2020|2020년ssppaaccee올해의ssppaaccee책ssppaaccee후보도서|ITssppaaccee모바일|YES24ssppaaccee올해의ssppaaccee책|국내도서|컴퓨터ssppaaccee공학|컴퓨터ssppaaccee교육|2019IT연말결산선정도서|개발자|분철|새해결심|집콕공부|컴공|파이썬|프로그래머|혼공|2019년ssppaaccee올해의ssppaaccee책ssppaaccee후보도서|ITssppaaccee모바일|YES24ssppaaccee올해의ssppaaccee책|국내도서|파이썬|프로그래밍ssppaaccee언어|분철|ITssppaaccee모바일|MSssppaacceeExcel(엑셀)|MSssppaacceePowerPoint(파워포인트)|MSssppaacceeWord(워드)|국내도서|오피스ssppaaccee활용|한글|ITssppaaccee모바일|OSand데이터베이스|국내도서|인공지능|자료구조and알고리즘|컴퓨터ssppaaccee공학|클라우드and빅데이터|파이썬|프로그래밍ssppaaccee언어|ITssppaaccee모바일|국내도서|모바일and태블릿andSNS|쇼핑몰and창업|앱가이드|인터넷ssppaaccee마케팅|인터넷ssppaaccee비즈니스|분철|ITssppaaccee모바일|국내도서|그래픽and디자인and멀티미디어|일러스트레이터|포토샵|1인창업|분철|온라인쇼핑몰창업|eto비즈니스|ITssppaaccee모바일|경제ssppaaccee경영|국내도서|마케팅and세일즈|쇼핑몰and창업|인터넷ssppaaccee마케팅|인터넷ssppaaccee비즈니스|인터넷비즈니스|2019IT연말결산선정도서|MS오피스|분철|사회초년생주목|새해결심|신입사원|일잘러업무스킬|집콕공부|칼퇴|ITssppaaccee모바일|MSssppaacceeExcel(엑셀)|국내도서|오피스ssppaaccee활용|분철|ITssppaaccee모바일|각국ssppaaccee

In [66]:
words = re.findall(r"\w+", totalTagsString)
counter = Counter(words)
dict(counter)

{'개발자': 61,
 '두잇': 16,
 '분철': 212,
 '새해결심': 11,
 '컴공': 44,
 '파이썬': 338,
 '프로그래밍': 57,
 'IT모바일': 2418,
 '국내도서': 2418,
 '프로그래밍언어': 781,
 '다시보는2020': 2,
 '2020년올해의책후보도서': 2,
 'YES24올해의책': 17,
 '컴퓨터공학': 960,
 '컴퓨터교육': 130,
 '2019IT연말결산선정도서': 29,
 '집콕공부': 4,
 '프로그래머': 10,
 '혼공': 3,
 '2019년올해의책후보도서': 5,
 'MSExcel': 85,
 '엑셀': 89,
 'MSPowerPoint': 54,
 '파워포인트': 88,
 'MSWord': 14,
 '워드': 14,
 '오피스활용': 185,
 '한글': 29,
 'OSand데이터베이스': 356,
 '인공지능': 264,
 '자료구조and알고리즘': 209,
 '클라우드and빅데이터': 250,
 '모바일and태블릿andSNS': 100,
 '쇼핑몰and창업': 40,
 '앱가이드': 16,
 '인터넷마케팅': 70,
 '인터넷비즈니스': 158,
 '그래픽and디자인and멀티미디어': 297,
 '일러스트레이터': 25,
 '포토샵': 63,
 '1인창업': 3,
 '온라인쇼핑몰창업': 5,
 'eto비즈니스': 35,
 '경제경영': 133,
 '마케팅and세일즈': 59,
 'MS오피스': 6,
 '사회초년생주목': 1,
 '신입사원': 3,
 '일잘러업무스킬': 7,
 '칼퇴': 4,
 '각국경제and경제사and전망': 6,
 '경제': 10,
 '경제전망': 4,
 '국제경제and금융and화폐': 6,
 '네트워크and해킹and보안': 144,
 '보안and해킹': 84,
 '블록체인': 21,
 '트렌드and미래예측': 21,
 '금융공학': 2,
 '데이터분석': 19,
 '퀀트': 2,
 'C언어': 7,
 '코딩': 22,
 'C': 128,
 'SQL': 2,
 '국가공인'

In [5]:
now_datetime = datetime.datetime.now()
start_datetime = now_datetime - datetime.timedelta(
    hours=now_datetime.hour,
    minutes=now_datetime.minute,
    seconds=now_datetime.second,
) 

In [6]:
now_datetime

datetime.datetime(2022, 1, 25, 0, 25, 1, 949867)

In [7]:
timezone.make_aware(now_datetime)

datetime.datetime(2022, 1, 25, 0, 25, 1, 949867, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

In [9]:
result = (
            Metadata.objects.filter(
                crawl_date__year=2021, crawl_date__month=2, crawl_date__day=24
            )
            .values("book__publisher")
            .annotate(
                sales_point_sum=Sum("sales_point"),
                sales_point_avg=Avg("sales_point"),
                rank_avg=Avg("rank"),
                number_of_book=Count("book__title"),
            )
            .order_by("-sales_point_sum")
        )

In [17]:
for item in result:
    print(item['book__publisher'], item['sales_point_sum'], item['sales_point_avg'], item['rank_avg'], item['number_of_book'])

한빛미디어 1068690 11132.1875 271.4270833333333 96
이지스퍼블리싱 600717 12781.212765957447 213.27659574468086 47
길벗 572160 10217.142857142857 270.2142857142857 56
제이펍 287340 8707.272727272728 308.2121212121212 33
위키북스 276903 7483.864864864865 279.2432432432432 37
인사이트(insight) 212475 9238.04347826087 270.04347826086956 23
영진닷컴 203229 5080.725 336.675 40
에이콘출판사 146145 4428.636363636364 315.6060606060606 33
생능출판사 117564 10687.636363636364 342.0 11
프리렉 103539 12942.375 296.75 8
한빛아카데미 103074 6063.176470588235 395.7647058823529 17
책만 94050 9405.0 268.9 10
오렌지미디어 93033 15505.5 292.0 6
성안당 87348 7940.727272727273 225.27272727272728 11
티더블유아이지 73350 73350.0 2.0 1
정보문화사 69516 4965.428571428572 315.64285714285717 14
비제이퍼블릭(BJ퍼블릭) 69252 4616.8 377.6 15
앤써북 68850 5737.5 333.75 12
한국데이터산업진흥원 43806 10951.5 136.0 4
도우출판 27381 13690.5 83.5 2
한국교육방송공사 25833 25833.0 34.0 1
책밥 25083 12541.5 186.0 2
e비즈북스 24546 12273.0 280.0 2
BOOKK(부크크) 22326 4465.2 345.6 5
루비페이퍼 21129 3018.4285714285716 409.2857142857143 7
시대고시기획